## Data Crawling

### 1. 빅케이스 계정정보 추출
빅케이스에 로그인할 계정 정보를 가져옵니다. 계정 정보는 현재 디렉토리에 `config.ini` 파일을 생성한뒤 아래와 같이 작성합니다.

```ini
[ACCOUNT]
email = 이메일
password = 비밀번호
```

In [2]:
import configparser as parser

properties = parser.ConfigParser()
properties.read("./config.ini")

email = properties['ACCOUNT']['email']
password = properties['ACCOUNT']['password']

### 2. 데이터 크롤링
계정 정보를 가지고 로그인한 뒤 데이터를 크롤링합니다.

**기본 상수 설정**
* `CHROMEDRIVER_PATH` : 셀레니움 라이브러리를 위한 크롬 드라이버 경로를 지정합니다.
* `COURT_ID` : URL 쿼리 중 'court' 부분에 해당하는 법원 ID입니다.
* `START_PAGNIATION_INDEX` : 시작 페이지 번호입니다.
* `END_PAGINATION_INDEX` : 종료 페이지 번호입니다. (최대 30)
* `OUTPUT_PATH` : 출력 파일이 저장될 폴더 경로입니다.
* `OUTPUT_FILENAME` : 출력 파일명입니다.

**법원 ID 리스트**
```text
1000 - 대법원

2100 - 서울고등법원
2200 - 대전고등법원
2300 - 대구고등법원
2400 - 부산고등법원
2500 - 광주고등법원
2600 - 수원고등법원

3101 - 서울중앙지방법원
3105 - 서울동부지방법원
3106 - 서울남부지방법원
3107 - 서울북부지방법원
3108 - 서울서부지방법원
3109 - 의정부지방법원
3111 - 인천지방법원
3121 - 춘천지방법원
3201 - 대전지방법원
3211 - 청주지방법원
3301 - 대구지방법원
3401 - 부산지방법원
3411 - 울산지방법원
3421 - 창원지방법원
3501 - 광주지방법원
3511 - 전주지방법원
3521 - 제주지방법원
3601 - 수원지방법원
```

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from tqdm import trange # 프로그레스바 출력을 위한 라이브러리
import urllib.parse # URL 디코딩을 위한 라이브러리
import pandas as pd
import os
import sys


#################
# 상수
#################
# 크롬 관련 상수
CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver' # 크롬 드라이버 위치
WINDOW_SIZE = "1920,1080" # 화면이 보이지 않는 CUI 환경이므로 가상으로 화면사이즈를 지정함

# 크롤링 관련 상수
COURT_ID = "2100"              # COURT ID (아무것도 입력하지 않으면 전체 판례를 가져옴[최대 300개])
START_PAGNIATION_INDEX = 1    # 시작 페이지 번호
END_PAGINATION_INDEX = 11      # 종료 페이지 번호
OUTPUT_PATH = "output/"     # 결과가 저장될 폴더 경로
OUTPUT_FILENAME = "data_{}.xlsx".format(COURT_ID)  # 결과 파일명


# Selenium 드라이버로 페이지에 접속하고 로딩할동안 기다림
def get(url):
    driver.get(url)
    WebDriverWait(driver, 10).until(lambda driver: driver.execute_script('return document.readyState') == 'complete')


# 결과 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

# 만약 동일한 이름의 파일이 있는 경우 프로그램 중단
if os.path.isfile(os.path.join(OUTPUT_PATH, OUTPUT_FILENAME)):
    print("Error : 동일한 이름을 가진 파일이 존재합니다.")
    print(os.path.join(OUTPUT_PATH, OUTPUT_FILENAME))
    sys.exit()


################
# Selenium 설정
################
# 크롬 드라이버 경로
chrome_service = Service(CHROMEDRIVER_PATH)

# 크롬 옵션
chrome_options = Options()
chrome_options.add_argument("--headless") # CUI 환경에서 필요한 옵션
chrome_options.add_argument('--no-sandbox') # CUI 환경에서 필요한 옵션
chrome_options.add_argument('--disable-dev-shm-usage') # 메모리 제한을 풀기 위한 옵션
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)


#####################
# 로그인
#####################
get("https://bigcase.ai/login")

# 계정 정보 입력
driver.find_element("id", "user-email").send_keys(email)
driver.find_element("id", "user-password").send_keys(password)

# 로그인
driver.find_element("css selector", ".bc-button").click()

# 로그인 될때까지 대기
WebDriverWait(driver=driver, timeout=10).until((lambda x: driver.current_url != "https://bigcase.ai/login"))


#####################
# 판례 데이터 크롤링
#####################
dataframe = pd.DataFrame() # 최종 데이터가 저장될 데이터프레임

for pagination in trange(START_PAGNIATION_INDEX, END_PAGINATION_INDEX + 1):
    get("https://bigcase.ai/search/case?q=%EB%8F%84%EB%A1%9C%EA%B5%90%ED%86%B5%EB%B2%95%EC%9C%84%EB%B0%98&page={}&case_type=5&period=3&level=&law=%EB%8F%84%EB%A1%9C%EA%B5%90%ED%86%B5%EB%B2%95&case_name=%EB%8F%84%EB%A1%9C%EA%B5%90%ED%86%B5%EB%B2%95&court={}".format(pagination, COURT_ID))

    # 해당 pagination에 해당하는 판례 url 리스트 생성
    urls = [ element.get_attribute("href") for element in driver.find_elements("css selector", ".search-list-card__open-new-window > a") ]

    for url in urls:
        splitted_url = url[19:].split("/")
        court = urllib.parse.unquote(splitted_url[1]) # 재판법원
        case_number = urllib.parse.unquote(splitted_url[2][:splitted_url[2].index("?") if "?" in splitted_url[2] else len(splitted_url[2])]) # 사건번호 - 만약 URL 뒤의 쿼리가 포함된 경우 쿼리를 삭제
        
        get(url) # 판례 페이지 접속

        # 판례 결과
        case_outcome = ""
        try:
            case_outcome = driver.find_element("css selector", ".case-content-info__outcome").text
        except NoSuchElementException:
            pass

        # 변경/폐기/파기된 판례의 경우 크롤링하지 않음
        try:
            if driver.find_element("css selector", ".detail-flag-tag").text == "변경/폐기/파기된 판례":
                continue
        except NoSuchElementException:
            pass
            

        # 사건 정보(여러개일 수 있음), 변호인, 원심판결(여러개일 수 있음), 판결선고 날짜
        case_infos, attorney_name, previous_cases, judgement_date = [[], ""] * 2
        for element in driver.find_elements("css selector", ".ci__info-row"):
            label = element.find_element("css selector", "i").text

            # 사건 정보 저장
            if label == "사건":
                case_infos = [ value.text for value in element.find_elements("css selector", ".ci__info-value") ]

            # 변호인 저장
            if label == "변호인":
                attorney_name = element.find_element("css selector", ".ci__info-value").text

            # 원심판결 저장
            if label == "원심판결":
                previous_cases = [ value.get_attribute("href") for value in element.find_elements("css selector", ".ci__info-value--link") ]

            # 판결선고 저장
            if label == "판결선고":
                judgement_date = element.find_element("css selector", ".ci__info-value").text
                
        # 주문, 이유(html 태그가 포함된 raw text)
        order, raw_reason = "", ""
        for element in driver.find_elements("css selector", ".case-detail-content__case-title-paragraph"):
            label = element.find_element("css selector", "h2").text

            # 주문 저장
            if label == "주 문":
                order = element.find_element("css selector", "i").text

            # 이유 저장
            if label == "이 유":
                for sub_element in element.find_elements("css selector", "div > *:is(i, span)"):
                    raw_reason += sub_element.get_attribute("innerHTML")

        
        # 유사판례 URL
        similar_cases = []
        if driver.find_elements("css selector", ".ai-similar-side__item-title"): # 유사판례가 있는 경우에만 실행
            for _ in range(int(driver.find_element("css selector", ".simple-pagination__page-number").text.split(" / ")[1])): # 유사 판례 리스트의 pagination의 마지막 페이지 번호를 가져옴
                similar_cases.extend([element.get_attribute("href") for element in driver.find_elements("css selector", ".ai-similar-side__item-title")])
                driver.find_element("css selector", ".simple-pagination__next-button").click() # 다음 페이지로 넘어감
            similar_cases = set(similar_cases) # 만약 네트워크 지연으로 동일한 유사 판례를 한번더 가져온 경우 중복된 판례들을 제거함
        
        # 임시 데이터프레임 생성 및 저장
        _dataframe = pd.DataFrame({
            "court" : court,
            "case_number" : case_number,
            "case_outcome" : case_outcome,
            "case_infos" : [case_infos],
            "attorney_name" : attorney_name,
            "previous_cases" : [previous_cases],
            "judgement_date" : judgement_date,
            "order" : order,
            "raw_reason" : raw_reason,
            "url" : url,
            "similar_cases" : [similar_cases]
        })
        dataframe = dataframe.append(_dataframe)


dataframe.to_excel(os.path.join(OUTPUT_PATH, OUTPUT_FILENAME), index=False) # excel 파일로 저장
print("Output file : " + os.path.join(OUTPUT_PATH, OUTPUT_FILENAME))

driver.close()
driver.quit()

100%|██████████| 11/11 [04:29<00:00, 24.54s/it]


Output file : output/data_2100.xlsx
